In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
## SkLearn# Collection of string documents

# # corpus = [
#      'this is the first document',
#      'this document is the second document',
#      'and this is the third one',
#      'is this the first document',
# # ]
data = pd.read_csv('sms+spam+collection/SMSSpamCollection', sep='\t', header=None, names=['label', 'text'])
data 

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [5]:
# sklearn feature names, they are sorted in alphabetic order by default.

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [6]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [7]:
# shape of sklearn tfidf vectorizer output after applying transform method.

skl_output.shape

(4, 9)

In [8]:
# sklearn tfidf values for first line of the above corpus.
# Here the output is a sparse matrix

print(skl_output[0]

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [9]:
# sklearn tfidf values for first line of the above corpus.
# To understand the output better, here we are converting the sparse output matrix to dense matrix and printing it.
# Notice that this output is normalized using L2 normalization. sklearn does this by default.

print(skl_output[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


In [10]:
# Write your code here.
# Make sure its well documented and readble with appropriate comments.
# Compare your results with the above sklearn tfidf vectorizer
# You are not supposed to use any other library apart from the ones given below

from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy

You should compare the results of your own implementation of TFIDF vectorizer with that of sklearns implemenation TFIDF vectorizer.

Sklearn does few more tweaks in the implementation of its version of TFIDF vectorizer, so to replicate the exact results you would need to add following things to your custom implementation of tfidf vectorizer:
Sklearn has its vocabulary generated from idf sroted in alphabetical order
Sklearn formula of idf is different from the standard textbook formula. Here the constant "1" is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions.  IDF(t)=1+loge1 + Total number of documents in collection1+Number of documents with term t in it. 
Sklearn applies L2-normalization on its output matrix.
The final output of sklearn tfidf vectorizer is a sparse matrix.

Steps to approach this task:
You would have to write both fit and transform methods for your custom implementation of tfidf vectorizer.
Print out the alphabetically sorted voacb after you fit your data and check if its the same as that of the feature names from sklearn tfidf vectorizer.
Print out the idf values from your implementation and check if its the same as that of sklearns tfidf vectorizer idf values.
Once you get your voacb and idf values to be same as that of sklearns implementation of tfidf vectorizer, proceed to the below steps.
Make sure the output of your implementation is a sparse matrix. Before generating the final output, you need to normalize your sparse matrix using L2 normalization. You can refer to this link https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html
After completing the above steps, print the output of your custom implementation and compare it with sklearns implementation of tfidf vectorizer.
To check the output of a single document in your collection of documents, you can convert the sparse matrix related only to that document into dense matrix and print it.

Note-1: All the necessary outputs of sklearns tfidf vectorizer have been provided as reference in this notebook, you can compare your outputs as mentioned in the above steps, with these outputs.
Note-2: The output of your custom implementation and that of sklearns implementation would match only with the collection of document strings provided to you as reference in this notebook. It would not match for strings that contain capital letters or punctuations, etc, because sklearn version of tfidf vectorizer deals with such strings in a different way. To know further details about how sklearn tfidf vectorizer works with such string, you can always refer to its official documentation.
Note-3: During this task, it would be helpful for you to debug the code you write with print statements wherever necessary. But when you are finally submitting the assignment, make sure your code is readable and try not to print things which are not part of this task.

In [21]:

#We will start by breaking our corpus into individual words
def split_list(sentence):
        result = []
    #Split each sentence in the list, and append to result list
        for s in sentence:
            result.append(s.split())
        return result
            
result = split_list(corpus)
print(result)

[['this', 'is', 'the', 'first', 'document'], ['this', 'document', 'is', 'the', 'second', 'document'], ['and', 'this', 'is', 'the', 'third', 'one'], ['is', 'this', 'the', 'first', 'document']]


In [17]:
#we will add this obtained list of words to a set so that we have only unique words
#reference: https://stackoverflow.com/questions/30083947/how-to-convert-list-of-lists-to-a-set-in-python-so-i-can-compare-to-other-sets
outer_list = [] 
def lists_to_list(nested_lists): 
    for el in nested_lists:
        if type(el) == list: 
            lists_to_list(el) 
        else: 
            outer_list.append(el)
    return outer_list
outer_all_list = lists_to_list(result)
print(outer_all_list)
outer_list = set(outer_all_list)
print("="*50)
print(outer_list)
result

['this', 'is', 'the', 'first', 'document', 'this', 'document', 'is', 'the', 'second', 'document', 'and', 'this', 'is', 'the', 'third', 'one', 'is', 'this', 'the', 'first', 'document']
{'second', 'and', 'one', 'the', 'first', 'this', 'document', 'is', 'third'}


[['this', 'is', 'the', 'first', 'document'],
 ['this', 'document', 'is', 'the', 'second', 'document'],
 ['and', 'this', 'is', 'the', 'third', 'one'],
 ['is', 'this', 'the', 'first', 'document']]

In [13]:
#sort the list alphabetically
print(outer_list)
sorted_list = sorted(outer_list)
print(sorted_list)

{'second', 'and', 'one', 'the', 'first', 'this', 'document', 'is', 'third'}
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [14]:
#a dicttionary for storing all words and their occurences in the corpus
numofwords = dict.fromkeys(sorted_list, 0)

for word in outer_all_list:
    numofwords[word] += 1 
    
print(numofwords)

{'and': 1, 'document': 4, 'first': 2, 'is': 4, 'one': 1, 'second': 1, 'the': 4, 'third': 1, 'this': 4}


In [ ]:
#computing the term frequency of words
def compute_tf(numofwords,outer_all_list):
    term_freq = {}
    len_of_list = len(outer_all_list)
    for word,count in numofwords.items():
        term_freq[word] = count/len_of_list
    return term_freq
term_freq = compute_tf(numofwords,outer_all_list)
print(term_freq)
outer_all_list

In [40]:
#computing count of words in each document
N = len(corpus)
idf_dict = dict.fromkeys(sorted_list,0)
for word,count in numofwords.items():
    for i in range(len(corpus)):
        for x in result[i]:
               # print (x)
                if x == word:
                    idf_dict[word] += 1
                    break
print(idf_dict)
result

{'and': 1, 'document': 3, 'first': 2, 'is': 4, 'one': 1, 'second': 1, 'the': 4, 'third': 1, 'this': 4}


[['this', 'is', 'the', 'first', 'document'],
 ['this', 'document', 'is', 'the', 'second', 'document'],
 ['and', 'this', 'is', 'the', 'third', 'one'],
 ['is', 'this', 'the', 'first', 'document']]

In [42]:
#computing IDF
idf_final = dict.fromkeys(sorted_list,0)
for word,val in idf_dict.items():
    idf_final[word] = 1 + (math.log((N+1)/(val+1))) 
print(idf_final)
sorted_list

{'and': 1.916290731874155, 'document': 1.2231435513142097, 'first': 1.5108256237659907, 'is': 1.0, 'one': 1.916290731874155, 'second': 1.916290731874155, 'the': 1.0, 'third': 1.916290731874155, 'this': 1.0}


4

In [39]:
#compute TF-IDF
tfidf = dict.fromkeys(sorted_list,0)
for word,val in idf_final.items():
    tfidf[word] = val * term_freq[word]
print(tfidf)

{'and': 0.08710412417609796, 'document': 0.2223897366025836, 'first': 0.13734778397872643, 'is': 0.18181818181818182, 'one': 0.08710412417609796, 'second': 0.08710412417609796, 'the': 0.18181818181818182, 'third': 0.08710412417609796, 'this': 0.18181818181818182}


In [20]:
type(tfidf)

dict